In [4]:
import sys
sys.path.insert(1, '../')
from FrenetFDA.utils.smoothing_utils import VectorBSplineSmoothing, local_polynomial_smoothing
import FrenetFDA.utils.visualization as visu
from FrenetFDA.utils.Frenet_Serret_utils import solve_FrenetSerret_ODE_SE
import numpy as np
from scipy.integrate import cumtrapz, solve_ivp

In [12]:
def generate_Y(X, gamma):
    W = gamma**2*np.eye(3)
    Y = X[1:] + np.random.multivariate_normal(np.zeros(3), W, size=(len(X)-1))
    return Y

N = 200
grid = np.linspace(0,1,N+1)


""" __________ Definition of exact function Theta ___________ """

curv = lambda s : 2*np.cos(2*np.pi*s) + 5
tors = lambda s : 2*np.sin(2*np.pi*s) + 1

def theta(s):
    if isinstance(s, int) or isinstance(s, float):
        return np.array([curv(s), tors(s)])
    elif isinstance(s, np.ndarray):
        return np.vstack((curv(s), tors(s))).T
    else:
        raise ValueError('Variable is not a float, a int or a NumPy array.')

In [13]:
Z = solve_FrenetSerret_ODE_SE(theta, grid, Z0=np.eye(4))
X = Z[:,:3,3]
gamma = 0.001
W = gamma**2*np.eye(3)
Y = generate_Y(X, gamma)

In [11]:
print(grid.shape, Y.shape)

(201,) (200, 3)


In [17]:
derivatives = local_polynomial_smoothing(4, Y, grid[1:], grid[1:], h=None, CV_optimization_h={"flag":True, "h_grid":np.array([1, 0.1, 0.01, 0.02, 0.03, 0.0001, 0.001]), "K":50})

error_h: [5.92662891e-05 1.39121118e-05 2.71944724e-05 2.95647089e-05
 2.70528662e-05 5.67982767e-01 5.67982767e-01]
Optimal smoothing parameter h find by cross-validation: 0.1


In [ ]:
visu.plot_3D([derivatives[0], X, Y], ['smooth_Y', 'X', 'Y'])

In [3]:
curv_noisy = curv(grid) + np.random.normal(0, 0.5**2, size=200)
print(curv_noisy.shape)

(200,)


In [11]:
basis_repres_1 = VectorBSplineSmoothing(1, 30)
# basis_repres.grid_search_optimization_regularization_parameter(grid, curv_noisy, np.array([0, 0.0001, 0.01]))

In [12]:
basis_repres_1.fit(grid, curv(grid), regularization_parameter=np.array([0.0001]))
smooth_curv_noisy = basis_repres_1.evaluate(grid)

In [13]:
confidence_limits = basis_repres_1.compute_confidence_limits(0.95)
lower_limit, upper_limit = confidence_limits(grid)

7.036279973434583e-05
[[7.03627997e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.03627997e-05 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.03627997e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.03627997e-05
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  7.03627997e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 7.03627997e-05]]


In [14]:
basis_repres_1.plot()

In [8]:
print(basis_repres.residuals_error)

0.06768781763932362


In [6]:
theta_noisy = theta(grid) + np.random.multivariate_normal(np.zeros(2), 0.5**2*np.eye(2), size=200)
theta_noisy[:,0] = theta(grid)[:,0]
print(theta_noisy.shape)

(200, 2)


In [4]:
basis_repres = VectorBSplineSmoothing(2, np.array([10,30]))
basis_repres.grid_search_optimization_regularization_parameter(grid, theta_noisy, np.array([[0,0], [0.001, 0.00001], [0.00001, 0.00001], [0, 0.00001]]))

Begin grid search optimisation with 4 combinations of parameters...
SSE [1.95048538e-04 4.40814168e+01]
df_lbda [10. 30.]
0.3050627887886738
SSE [ 0.22975412 45.41565876]
df_lbda [ 7.15509405 21.52730049]
0.286397844460856
SSE [7.27422720e-04 4.54156588e+01]
df_lbda [ 9.82883315 21.52730049]
0.2851662699252919
SSE [1.95048538e-04 4.54156588e+01]
df_lbda [10.         21.52730049]
0.28516332773519254
Optimal regularization parameter selected by grid search optimisation:  [0.e+00 1.e-05]


array([0.e+00, 1.e-05])

In [11]:
basis_repres = VectorBSplineSmoothing(1, 30)
basis_repres.grid_search_optimization_regularization_parameter(grid, theta_noisy[:,0], np.array([0, 0.1, 0.001, 0.00001]))

Begin grid search optimisation with 4 combinations of parameters...
Optimal regularization parameter selected by grid search optimisation:  0.0


0.0

In [4]:
basis_repres = VectorBSplineSmoothing(2, 30)

In [9]:
basis_repres.fit(grid, theta_noisy, regularization_parameter=np.array([0.0001,0.1])) 
smooth_theta_noisy = basis_repres.evaluate(grid)

In [10]:
confidence_limits = basis_repres.compute_confidence_limits(0.95)
lower_limit, upper_limit = confidence_limits(grid)
basis_repres.plot()

[7.03627997e-05 6.41119855e-01]
[[7.03627997e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.41119855e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.03627997e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 6.41119855e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  7.03627997e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 6.41119855e-01]]


In [19]:
basis_repres = VectorBSplineSmoothing(2, 30)
basis_repres.fit(grid, theta_noisy, regularization_parameter=np.array([0,0.1]))
smooth_theta_noisy = basis_repres.evaluate(grid)
confidence_limits = basis_repres.compute_confidence_limits(0.95)
lower_limit, upper_limit = confidence_limits(grid)
basis_repres.plot()